# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [60]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [61]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao joao da barra,-21.6403,-41.0511,295.49,66,96,5.33,BR,1682055598
1,1,west island,-12.1568,96.8225,302.14,74,79,7.72,CC,1682055599
2,2,stanley,54.8680,-1.6985,281.08,88,59,5.66,GB,1682055599
3,3,papatowai,-46.5619,169.4708,284.88,91,78,0.26,NZ,1682055600
4,4,hawaiian paradise park,19.5933,-154.9731,298.88,93,75,3.60,US,1682055600


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [65]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600, 
    size = "Humidity",
    scale = 1.5,
    color = "City"
    )
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [78]:
# Narrow down cities that fit criteria and drop any results with null values
#perferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <=27),
                                      #(city_data_df["Max Temp"] >= 21)], (city_data_df["Wind Speed"] >= 5 m/s), (city_data_df["Cloudiness"] = 0).dropna()
perferred_cities =city_data_df.loc[(city_data_df['Max Temp'] < 27) &
                             (city_data_df['Max Temp'] > 21) &
                             (city_data_df['Wind Speed'] < 4.5) &
                             (city_data_df['Cloudiness'] < 0) 
                             ]
perferred_cities= preferred_cities.dropna() 


# Drop any rows with null values
#perferred_cities = perferred_cities.dropna()

# Display sample data
perferred_cities.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao joao da barra,-21.6403,-41.0511,295.49,66,96,5.33,BR,1682055598
1,1,west island,-12.1568,96.8225,302.14,74,79,7.72,CC,1682055599
2,2,stanley,54.8680,-1.6985,281.08,88,59,5.66,GB,1682055599
3,3,papatowai,-46.5619,169.4708,284.88,91,78,0.26,NZ,1682055600
4,4,hawaiian paradise park,19.5933,-154.9731,298.88,93,75,3.60,US,1682055600
5,5,mount isa,-20.7333,139.5000,304.02,29,0,5.14,AU,1682055601
6,6,uturoa,-16.7333,-151.4333,301.65,78,100,8.16,PF,1682055601
7,7,bull savanna,17.8868,-77.5902,295.50,85,5,2.35,JM,1682055601
8,8,vila franca do campo,37.7167,-25.4333,287.38,79,68,4.88,PT,1682055602
9,9,pasuquin,18.3328,120.6177,305.12,70,40,4.63,PH,1682055603


### Step 3: Create a new DataFrame called `hotel_df`.

In [66]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perferred_cities[["City", "Country", "Humidity", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Humidity,Lat,Lng,Hotel Name
0,sao joao da barra,BR,66,-21.6403,-41.0511,
1,west island,CC,74,-12.1568,96.8225,
2,stanley,GB,88,54.8680,-1.6985,
3,papatowai,NZ,91,-46.5619,169.4708,
4,hawaiian paradise park,US,93,19.5933,-154.9731,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [68]:
# Set parameters to search for a hotel (do i use 'type' or 'categories' for hotel)
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
sao joao da barra - nearest hotel: Pousada Mediterrâneo
west island - nearest hotel: Cocos Village Bungalows
stanley - nearest hotel: Hotel 52
papatowai - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
uturoa - nearest hotel: Hawaiki Nui hotel
bull savanna - nearest hotel: Junction Guest House
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
pasuquin - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
toliara - nearest hotel: Ambary
utrik - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
prince george - nearest hotel: Travelodge by Wyndham Prince George
port-aux-francais - nearest hotel: Keravel
bubaque - nearest hotel: Chez Julio
queenstown - nearest hotel: Queens Hotel
punta arenas - nearest hotel: Hotel Lacolet
st. john's - nearest hotel: Alt Hotel
polyarnyy - nearest hotel: No hotel found
midvagur - nearest hotel

,City,Country,Humidity,Lat,Lng,Hotel Name
0,sao joao da barra,BR,66,-21.6403,-41.0511,Pousada Mediterrâneo
1,west island,CC,74,-12.1568,96.8225,Cocos Village Bungalows
2,stanley,GB,88,54.8680,-1.6985,Hotel 52
3,papatowai,NZ,91,-46.5619,169.4708,No hotel found
4,hawaiian paradise park,US,93,19.5933,-154.9731,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)